In [1]:
from threading import Lock, Thread
import time
import numpy as np
from modules.wavelengthmeter import WavelengthMeter
from modules.usb_dao import USB_DAO
from modules.udp_discover_secure import get_key_from_password, Discovery
from modules.lock_server import web_lock
from modules.lock_controller import config_helper
from modules.lock_controller import Controller
from modules.TopticaLaserController import *
from modules.FiberSwitchCommunication import FiberSwitch
from modules.PIDTestFunctions import SineTestFunction
from toptica.lasersdk.dlcpro.v2_2_0 import DLCpro, NetworkConnection, SerialConnection
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

def data2db(paras, data):
    bucket = "wavemeter"
    client = InfluxDBClient(url="http://10.5.78.176:8086", token="yk11EXU-Dka5f7jQyUmilprZdf1YPUFrFxwHkITBYked0fjOynLjdw_tMYcu51Ul6ywCjCpmLQuMqm0ZSWg2kA==", org="BaLi")
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
    p = Point("laser_status").tag("wavemeter", "highfinesse").field(paras, data)
    write_api.write(bucket=bucket, record=p)
    client.close()

Do_CH1 = 0 # BDfile:///C:/Users/bali/Desktop/FiberSwitch/wavemeter-lock-project/web-lock/modules/wavelengthmeter.py#line=47
Do_CH2 = 5
Do_CH3 = 1 # Photo
Do_CH4 = 2

Ao_CH1 = 0 # BD
Ao_CH2 = 1
Ao_CH3 = 2 # Photo
Ao_CH4 = 3

#### Setpoint Frequencies of Lasers
#493nm:607.426044
#553nm:541.433000
#615nm:487.99008
#650nm:461.312470

##initialize Fiber Switch and connect to Laser Controllers
dev = USB_DAO()
FiberSwitch = FiberSwitch(port = "COM1", baudrate = 9600)


## Assign Channels of the Fiber Switch to the Wavemeter Channels --> implement in GUI later
SwitchChannels = {
    'WMCH1': "A",
    'WMCH2': "B",
    'WMCH3': "C",
    'WMCH4': "D"
}

#
mutex = Lock()

# Thread for automatic Calibration of the WM
def periodic_calibration(wvm, lck):
    def calibrate_loop():
        while True:
            CalibrationSettings = lck.calibration_settings
            ReferenceFrequency = CalibrationSettings['wm_calibration_frequency']
            ################################ hardcoded for safety reasons, later just in config.json  file
            ReferenceFrequency = 461.312470
            ############################
            DeltaFreqMax = 0.02 ## 20 MHz    
            interval_sec = CalibrationSettings['wm_calibration_interval'] if CalibrationSettings['wm_calibration_interval'] > 0 else 3600
            time.sleep(interval_sec) # wait time interval until next calibration
            lck.cntrl.pause() ############# pause control loop while calibrating
            try:
                pass
                #wvm.Calibration(Type = 2, unit = 2 , value = ReferenceFrequency, channel = 1)
                # Type = 2 : cOther
                # unit = 2:   cReturnFrequency
                # value = Reference Frequency in THz
                # channel = 1
            except Exception as e:
                pass
            lck.cntrl.resume()
    Thread(target=calibrate_loop, daemon=True).start()


def CheckReferenceLock(wvm , lck):
    def LockCheckLoop():
        while True:
            Channel = "650 nm"
            ReferenceFrequency = 461.312470
            ############################
            DeltaFreqMax = 0.000005 ## 5 MHz maximum deviation   
            #interval_sec = CalibrationSettings['wm_calibration_interval'] if CalibrationSettings['wm_calibration_interval'] > 0 else 3600
            interval_sec = 600
            time.sleep(interval_sec) # wait time interval until next calibration
            lck.cntrl.pause() ############# pause control loop while calibrating
            try:
                time.sleep(5)
                wvm.SetActiveChannel(channel=2, port=1)
                ActiveChannelCondition = (wvm.GetActiveChannel() == (2, 1))
                while ActiveChannelCondition == False:
                    channel, port = wvm.GetActiveChannel()
                wvm.SetExposureMode(True)
                time.sleep(1)
                Freq650Laser = wvm.GetFrequency2()
                try:
                    data2db('650nm', Freq650Laser)
                    data2db('650_power', read_power())
                except:
                    pass
                lck.cntrl.latest_values[Channel] = Freq650Laser
                FreqDeviation = abs(Freq650Laser - ReferenceFrequency)
                if FreqDeviation > DeltaFreqMax:
                    lck.cntrl.ReferenceLockState = False
                else:
                    lck.cntrl.ReferenceLockState = True
                wvm.SetActiveChannel(channel=1, port=1)
                ActiveChannelCondition = (wvm.GetActiveChannel() == (1, 1))
                while ActiveChannelCondition == False:
                    channel, port = wvm.GetActiveChannel()
                wvm.SetExposureMode(False)
                time.sleep(1)
            except Exception as e:
                print(e)
                pass
            lck.cntrl.resume()
    Thread(target=LockCheckLoop, daemon=True).start()


def MonitorWavemeter(wvm, lck):
    def WMMonitoringLoop():
        while True:
            try:
                time.sleep(0.1)
                Freq = wvm.GetFrequency()
                lck.cntrl.LastWMValue = Freq
                # hier könntest du z.B. lck.cntrl.latest_values['WVM'] = Freq eintragen
            except Exception as e:
                print(e)
                pass
    Thread(target=WMMonitoringLoop, daemon=True).start()


def manual_switch_control(fiber_switch, controller):
    print("\n--- Starting Manual Fiber Switch Control")
    
    # Regelung pausieren
    print("Pausing Laser Lock.")
    controller.pause()
    
    try:
        while True:
            cmd = input("➡️  enter channel (A, B, C, D) oder press 'q' to end: ").strip().upper()
            if cmd == 'Q':
                break
            elif cmd in ['A', 'B', 'C', 'D']:
                print(f"🔁 Umschalten auf Kanal {cmd}...")
                fiber_switch.SendCommand(cmd)
            else:
                print("⚠️  Ungültige Eingabe. Erlaubt: A, B, C, D, q")
    except KeyboardInterrupt:
        print("\n⛔ Unterbrochen durch Benutzer")
    
    print("▶️  Regelung wird fortgesetzt...")
    controller.resume()
    print("--- Manuelle Kontrolle beendet ---\n")


class WM_SELCTOR:
    switch_dur = .1
    def __init__(self, OutputFunction, ChannelName, LaserController, DACAnalogOut, expt_1=8, expt_2=5, **kwargs):
        super().__init__(**kwargs)
        self.OutputFunction = OutputFunction
        self.ChannelName = ChannelName
        self.LaserController = LaserController
        self.DACAnalogOut = DACAnalogOut
        self.expt_1 = expt_1
        self.expt_2 = expt_2
        self.ConfigHelper = config_helper()

        try:
            last_output = self.ConfigHelper.get(self.ChannelName, 'last_piezo_output')
            lck.cntrl.latest_piezo_values[self.ChannelName] = last_output
            if last_output is not None:
                print(f"[Init] Setting initial output for {self.ChannelName} to {last_output}")
                self.OutputFunction(last_output, self.LaserController, self.DACAnalogOut)
        except Exception as e:
            print(f"[Init] No previous output found for {self.ChannelName}: {e}")
        
    def func_read(self):  
        ## Function for Reading out Wavemeter
        with mutex:
            ConfigHelper = config_helper()
            SingleChannelCondition = hasattr(lck.cntrl.pid_dict[self.ChannelName], 'SingleChannelMode') and lck.cntrl.pid_dict[self.ChannelName].SingleChannelMode

            Exposure = lck.cntrl.get_wm_exposure(self.ChannelName)
            t_wait = 0.2 # max([Exposure, t_exp_1])/1000.+0.05

            FiberSwitch.SendCommand(SwitchChannels[self.ChannelName])
            time.sleep(t_wait) 

            time.sleep(t_wait)
            # for i in range(3): # why takes three times?
            try:
                freq = wvm.GetFrequency()
                data2db(self.ChannelName, freq)
                data2db(self.ChannelName+'_power', read_power())
                data2db('temperature', read_temp())
                data2db('pressure', read_pres())
            except:
                pass
            
            #wvm.SetExposure(0, num=1, ccd_arr=1)
            # Warning if WM under- overexposed (laser unlocked automatically, if wavelength is out of range)
            state = "WM Readout Working"
            if freq <= 0:
                if freq == -4:
                    state = "Overexposed"
                if freq in (0, -1, -2, -3):
                    state = "Underexposed"
                if freq < -4: state = "Unknown WM Reading Error" 
                #print ("WM Reading state: ", state)
            lck.cntrl.set_wm_reading_state(self.ChannelName, state)
            time.sleep(Exposure/1000+0.04)
        return freq
        
    def func_write(self, new, last): #################### Output Function
        #dev.aout(channel=self.Ao, value=new)
        SetPiezoVoltage = self.OutputFunction(new, self.LaserController, self.DACAnalogOut)
        try:
            ConfigHelper = config_helper()  # Instanz des Config-Helpers
            ConfigHelper.set(self.ChannelName, 'last_piezo_output', SetPiezoVoltage)
            # Optional: sofort speichern, falls ConfigHelper nicht automatisch speichert
            ConfigHelper.save()
            lck.cntrl.latest_piezo_values[self.ChannelName] = SetPiezoVoltage
            try:
                data2db(self.ChannelName+'_err_output', float(ConfigHelper.get(self.ChannelName, 'last_piezo_output')))
            except:
                print(type(ConfigHelper.get(self.ChannelName, 'last_piezo_output')))
                pass
        except Exception as e:
            print(f"[Error] Failed to save output for channel {self.ChannelName}: {e}")
            pass
        return new

def read_temp():
    return wvm.GetTemperature()

def read_pres():  
    return wvm.GetPressure()

def read_power():
    return wvm.GetPower() 

daq devices:   [<mcculw.structs.DaqDeviceDescriptor object at 0x00000207511B4550>]
0 USB-3112
No Device Found
DAC Range:  1
Switch Port:   COM4
Fiber Switch connection set up.


## Run the Lock Server and add Channels

In [ ]:
http_port = 80
host = '10.5.78.115'
#host = '127.0.0.1'
password = 'balibali24'
key = get_key_from_password(password)
dis = Discovery(key=key)
name = dis.hostname()
info = '%s:%i'%(name, http_port)
dis.echo_start(info=info)

# connect to DLCpro Laser Controller
IP='192.168.0.9'
PORT = 'COM5'
command_line_port=1998
#with DLCpro(NetworkConnection(IP)) as dlc:
with DLCpro(SerialConnection(PORT)) as dlc:
    # initialize Web server
    lck = web_lock(sampling=.075)
    # Connect to Wavemeter, initialize periodic Calibration every hour
    wvm = WavelengthMeter()
    wvm.start()
    wvm.SetAutoCalMode(0) #deactivate AutoCalibration to not interrupt the locking process
    wvm.SetActiveChannel(channel=1, port=1)
    #periodic_calibration(wvm, lck) # Wavemeter Calibration loop thread
    CheckReferenceLock(wvm, lck) ### Check if 650nm Reference Laser is in Lock
    MonitorWavemeter(wvm, lck)
    
    #Assign Laser Controllers and Switch Channels, replace with DAC analog output channels
    #wm_ch1 = WM_SELCTOR(LaserControllerCh1.SetPiezoVoltage, 'WMCH1')
    wm_ch2 = WM_SELCTOR(SetPiezoVoltageDL110, 'WMCH2',  LaserController = dev, DACAnalogOut = 1)
    wm_ch3 = WM_SELCTOR(SetPiezoVoltage, 'WMCH3', LaserController = dlc, DACAnalogOut = None)  ## 493 nm laser (lock already works)
    wm_ch4 = WM_SELCTOR(SetPiezoVoltageDL110, 'WMCH4', LaserController = dev, DACAnalogOut = 0)
    
    ## Add up to four Channels for Laser Locking
    ## active: if True, the channel is selected in the switching process if False: Channel is skipped
    #lck.add('WMCH1', wm_ch1.func_read, wm_ch1.func_write, 
     #       active=True, lock_type=1, tracelen=1000, 
      #      unit_input='Frequency (THz)', unit_output='Output (V)')
    lck.add('WMCH2', wm_ch2.func_read, wm_ch2.func_write, active=True, lock_type=1, tracelen=1000, 
           unit_input='Frequency (THz)', unit_output='Output (V)')
    lck.add('WMCH3', wm_ch3.func_read, wm_ch3.func_write, 
            active=True, lock_type=1, tracelen=1000, 
            unit_input='Frequency (THz)', unit_output='Output (V)')
    lck.add('WMCH4', wm_ch4.func_read, wm_ch4.func_write,  
            active=True, lock_type=1, tracelen=1000, 
            unit_input='Frequency (THz)', unit_output='Output (V)')

    # Run PID Controller
    lck.run(host=host, port=http_port, debug=False)
    wvm.stop()
    dis.echo_stop()

Key:  b'7E-XiQq-Zv7ojHK9ICY2Xh8Pd2t5MlpvAwsUF_JFB_0='
Listen on :37020
[Init] Setting initial output for WMCH2 to 3.7967016010052235
[Init] Setting initial output for WMCH3 to 35
[Init] Setting initial output for WMCH4 to 0.6934872218301144
Controller Running,  1770624016.3901043
